In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 2s (115 kB/s)
Reading package li

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok|2015-08-31 00:00:00|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...| 

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [ ]:
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [ ]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [ ]:
#Step1: Filter on the total_votes count equal to or greater than 20
vine_df1 = vine_df.filter("total_votes >=20")
vine_df1.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R3EM8C9CKA6GPK|          2|           41|        115|   N|                N|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|     

In [ ]:
#Step2: Filter on the percentage of helpful_votes equal to or greater than 50%
vine_df2 = vine_df1.filter("helpful_votes/total_votes >= 0.5")
vine_df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [ ]:
#Step3: Filter on the reviews written as part of the Vine program (paid)
vine_df3 = vine_df2.filter(vine_df2["vine"] == "Y")
vine_df3.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|
| RGXLFTR31J561|          5|           29|         34|   Y|                N|
|R2ZS3WCJYUDQH0|          3|          101|        115|   Y|                N|
|R31771VBPRDVJS|          4|           75|         83|   Y|                N|
| R8O8KM73GO190|          5|           21|         28|   Y|                N|
| RPU1GURA1NOT0|          5|           33|         35|   Y|     

In [ ]:
#Step4: Filter on the reviews written not as part of the Vine program (unpaid)
vine_df4 = vine_df2.filter(vine_df2["vine"] == "N")
vine_df4.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [ ]:
#Step5: Determine the total #reviews, the # of 5-star reviews, the % of 5-star reviews for the two types of review (paid vs unpaid).
# 5a. Total number of Vine reviews (paid)  
total_review_paid = vine_df3.count()
total_review_paid

607

In [ ]:
#5b. Total number of 5-star Vine reviews (paid)
paid_5star = vine_df3.filter("star_rating == 5").count()
paid_5star

257

In [ ]:
#5c. The percentage of 5-star Vine reviews (paid)
percent_paid_5star = (paid_5star/total_review_paid) *100
percent_paid_5star

42.33937397034596

In [ ]:
# 5d. Total number of non-Vine reviews (unpaid)  
total_review_unpaid = vine_df4.count()
total_review_unpaid

50522

In [ ]:
#5e. Total number of 5-star non-Vine reviews (unpaid)
unpaid_5star = vine_df4.filter("star_rating == 5").count()
unpaid_5star

25220

In [ ]:
#5f. The percentage of 5-star non-Vine reviews (unpaid)
percent_unpaid_5star = (unpaid_5star/total_review_unpaid) *100
percent_unpaid_5star

49.91884723486798